In [ ]:
#IMPORTS
import random
import csv
import heapq
import matplotlib.pyplot as plt
from zipfile import ZipFile
import os
from glob import glob
import cv2
import numpy as np
import random
import math

In [ ]:
# #QUESTION 1 DATA READ AND NORMALIZE 
data_tr=[]
data_tt=[]

#READS DATA 
with open('health_data.csv','r') as file:
  csvf= csv.DictReader(file)
  data=[]
  for row in csvf:
    data.append(row)
 
  ll=random.sample(range(700),490)
  for i in range(700):
    if i in ll:
      data_tr.append([int(x) for x in data[i].values()])
    else:
      data_tt.append([int(x) for x in data[i].values()])
  
data_ttt = np.array(data_tt) 
data_trr = np.array(data_tr)

#NORMALIZATION DATA

normalize  = np.zeros((len(data_ttt[0])-1))
data_tr=dict()
data_tt=dict()
possible_outcomes = set()
for i in data_ttt:
  possible_outcomes.add(i[len(i)-1])
  for j in range(len(i)-1):
    if normalize[j] < i[j]:
      normalize[j] = i[j]
for i in possible_outcomes:
  data_tr[i] = []
  data_tt[i] = []
for i in data_ttt:
  data_tt[i[len(i)-1]].append( i[:-1]/normalize  )

for i in data_trr:
  data_tr[i[len(i)-1]].append( i[:-1]/normalize )

for i in possible_outcomes:
  data_tr[i] = np.array(data_tr[i])
  data_tt[i] = np.array(data_tt[i])

X = []
y = []

def sigmoid(X):
  return 1/(1+np.exp(-1*X))

def prepare_data_for_loget():
  for i in data_tr.items():
    for j in i[1]:
      X.append(j)
      y.append(i[0])

#prior
dicto = {}
def prior():
  nu_0=len(data_tr[0])
  nu_1=len(data_tr[1])
  total=nu_0+nu_1
  p_0=nu_0/total
  p_1=nu_1/total
  return p_0,p_1
dicto[0],dicto[1] = prior()
prepare_data_for_loget()

X = np.array(X)
y = np.array(y)
# Done :D

In [ ]:
# fits gaussian on a given dataset
def fit_gaussian(data_tr):
  data_tr = np.array(data_tr)
  return (np.reshape(np.mean(data_tr,axis=0),(3,1)),np.cov(data_tr.T,rowvar=1))

# Trains Gaussian
dict1 = {}
for i in data_tr.items():
  dict1[i[0]] = fit_gaussian(i[1])

In [ ]:
# Evaluates Gaussian and returns its logarithm
e = math.exp(1)
def gaussian(x,sigma,u,n):
  x = np.reshape(x,(len(x),1))
  const =  1/math.sqrt( ( ((2*math.pi)**n)*abs(np.linalg.det(sigma)) ) ) 
  other_term = ((np.transpose(x-u))@(np.linalg.pinv(sigma)))@(x-u)
  return math.log(const,e)  + -0.5*other_term

# predicts label given a data point 
def predict(x,th):
  p0 = dicto[0]*math.exp(gaussian(x,dict1[1][1],dict1[0][0],len(x)))
  p1 = dicto[1]*math.exp(gaussian(x,dict1[1][1],dict1[1][0],len(x)))
  if p1/p0 > th:
    return 1
  else:
    return 0
    
# evaluate model at entire test set and returns relevant results
def check(th):
  right = 0
  l = 0
  tp = 0 
  tn = 0
  fp = 0
  fn = 0
  for i in data_tt.items():
    for j in i[1]:
      l+=1
      pre = int(predict(j,th))
      if i[0] == pre and i[0]== 1:
        tp+=1
        right+=1
      elif i[0] == pre and i[0] == 0:
        tn+=1
        right+=1
      elif i[0] == 1 and pre == 0:
        fn+=1
      elif i[0] == 0 and pre == 1:
        fp+=1
  prescion = tp/(tp+fp)
  recall = tp/(tp+fn)
  f1 = 2/(1/recall+1/prescion)
  accuracy = right/l
  tpr = 1
  fpr = 1
  if tp + fn!=0:
    tpr = tp/(tp+fn)
  if fp+tn!=0:
    fpr = fp/(fp+tn)
  return accuracy,prescion,f1,recall,tpr,fpr,tp,fp,tn,fn
print(check(1))

In [ ]:
# creates unimodal gaussian models for a given dataset :(
def sasta_fit_gaussian(data_tr):
  data_tr = np.array(data_tr)
  return np.mean(data_tr),np.var(data_tr)
dict1 = {}

In [ ]:
#naive bayes model
e = math.exp(1)
# evalevates unimodal gaussian 
def sasta_gaussian(x,sigma,u):
  return (1/math.sqrt(math.pi*2*sigma))*math.exp(-0.5*((x-u)*(x-u)/sigma)) 

# devlaration issue reolved :(
possible_outcomes = list(possible_outcomes)

# trains navie bayes for each feature for each parameter
def naive_bayes(data_tr):
  n = 3
  models = np.zeros((3,2,2))
  for i in range(n):
    for j in possible_outcomes:
      models[i][j][0],models[i][j][1] = sasta_fit_gaussian( (data_tr[j].T[i]) ) 
  return models

models = naive_bayes(data_tr)

#Evaluates Naive bayes over entier test set and returns relevant results
def eval(data_tt,th):
  right = 0
  l = 0
  tp = 0 
  tn = 0
  fp = 0
  fn = 0
  for i in data_tt.items():
    for j in i[1]:
      l+=1
      pre = int(predict_n(j,th))
      if i[0] == pre == 1:
        tp+=1
        right+=1
      elif i[0] == pre == 0:
        tn+=1
        right+=1
      elif i[0] == 1:
        fn+=1
      elif i[0] == 0:
        fp+=1
  prescion = tp/(tp+fp)
  recall = tp/(tp+fn)
  f1 = 2/(1/recall+1/prescion)
  accuracy = right/l
  tpr = tp/(tp+fn)
  fpr = fp/(fp+tn)
  return accuracy,prescion,f1,recall,tpr,fpr,tp,tn,fp,fn

# predicts a label for a point naive bayes
def predict_n(x,th):
  probab = []
  for i in range(len(possible_outcomes)):
    temp = dicto[i]
    for j in range(3):
      temp*=sasta_gaussian(x[j],models[j][i][1],models[j][i][0])
    probab.append(temp)
  if probab[1]/probab[0]>th:
    return 1
  else:
    return 0
print(eval(data_tt,1))

In [ ]:
# EM algoritm for GMM 
e =math.exp(1)
def GM(data,iteration,k):
  data = np.array(data)
  dimension = len(data[0])
  n = len(data)
  pk = np.array( [1/k]*k )
  mu = np.array( np.random.uniform(low=0.001,high=0.002, size=(k,dimension,1)) )
  sigma = np.array( np.random.uniform(low=0.001,high=1000, size=(k,dimension,dimension)) )
  log_liklihood = -1e18
  for iter in range(iteration):
    tmu = np.zeros((k,dimension,1))
    tsigma = np.zeros((k,dimension,dimension))
    tpk = np.zeros((k,1))
    global_sum = 0 

    dp = np.zeros((n,k))
    new_log_liklihood = 0

    #E - Step
    for i in range(n):
      normal = 0
      for j in range(k):
        oo = np.reshape(data[i],(data[i].shape[0],1))
        dp[i][j] = pk[j]*math.exp(gaussian(oo,sigma[j],mu[j],dimension))
        normal += dp[i][j]
      dp[i]/=normal
      new_log_liklihood += math.log(normal,e)
      for j in range(k):
        global_sum+=dp[i][j]
        tpk[j]+=dp[i][j]
      
    # M - step
    for i in range(n):
      for j in range(k):
        oo = np.reshape(data[i],(data[i].shape[0],1))
        factor =  (dp[i][j]/tpk[j])
        tmu[j] += factor*oo
        oj = (oo-mu[j])@(np.transpose(oo-mu[j]))
        tsigma[j] += factor*oj

    for i in range(k):
      tpk[i]/=global_sum

    #Exchange 
    pk = tpk.copy()
    sigma = tsigma.copy()
    mu = tmu.copy()
    if abs(new_log_liklihood-log_liklihood)<1e-6:
      break
    
    log_liklihood = new_log_liklihood
  return [pk,mu,sigma]  

In [ ]:
#trains GMM model
pp = dict()
def gmmtrain(k):
    global pp
    pp = dict()
    for i in data_tr.items():
        pp[i[0]] = GM(i[1],10000,k)  

In [ ]:
# predicts a label of a point using gmm
def predict_gmm(x,k,th):
  probab = []
  for i in pp.items():
    sum = 0
    for j in range(k):
      sum += i[1][0][j]*math.exp(gaussian(x,i[1][2][j],i[1][1][j],len(x)))
    probab.append(dicto[i[0]]*sum)
  if probab[1]/probab[0] > th:
    return 1
  else:
    return 0

# evalueavtes GMM over all test set
def check_gmm(k,th):
  right = 0
  right2 = 0
  l = 0
  l2 = 0
  tp = 0 
  tn = 0
  fp = 0
  fn = 0
  for i in data_tr.items():
    for j in i[1]:
      l2+=1
      pre = int(predict_gmm(j,k,th))
      if i[0] == pre:
        right2+=1
    
  for i in data_tt.items():
    for j in i[1]:
      l+=1
      pre = int(predict_gmm(j,k,th))
      if i[0] == pre == 1:
        tp+=1
        right+=1
      elif i[0] == pre == 0:
        tn+=1
        right+=1
      elif i[0] == 1:
        fp+=1
      elif i[0] == 0:
        fn+=1
  prescion = tp/(tp+fp)
  recall = tp/(tp+fn)
  f1 = 2/(1/recall+1/prescion)
  accuracy = right/l
  tpr = tp/(tp+fn)
  fpr = fp/(fp+tn)
  train_loss = right2/l2
  return accuracy,prescion,f1,recall,tpr,fpr,train_loss,tp,tn,fp,fn
gmmtrain(1)
print(check_gmm(1,1))

In [ ]:
#GENERAL KNN 

def distance(a,b):
  return (int(a)-int(b))*(int(a)-int(b))

# predicts label using knn model given test data,point,k and threshhold
def knn(data,point,k,th): 
  n = len(data)
  o = len(point)
  heap = []
  for i in range(n):
    dist = 0
    for j in range(o):
      dist += distance(data[i][j],point[j])
    heap.append((dist,data[i][o]))
  heapq.heapify(heap)
  probab = [0,0]
  for i in range(k):
    temp = heapq.heappop(heap)[1]
    probab[temp]+=1
  if probab[0]==0 or probab[1]/probab[0] > th:
    return 1
  else:
    return 0

# evalueavtes knn over all test set
def knn_caller(data,test,k,th):
  n = len(test)
  o = len(data[0])
  tp = 0
  tn = 0
  fp = 0
  fn = 0
  right = 0
  for i in range(n):
    lis = []
    for j in range(o-1):
      lis.append(test[i][j])
    temp = knn(data,lis,k,th)
    if temp==test[i][o-1]:
      right+=1
    if temp==1 and test[i][o-1]==1:
      tp+=1
    elif temp==0 and test[i][o-1]==0:
      tn+=1
    elif temp!=0 and test[i][o-1]==0:
      fn+=1
    else:
      fp+=1
  prescion = tp/(tp+fp)
  recall = tp/(tp+fn)
  f1 = 2/(1/recall+1/prescion)
  accuracy = right/n
  tpr = 1
  fpr = 1 
  if tp + fn != 0: 
    tpr = tp/(tp+fn)
  if tp + fn != 0:
    fpr = fp/(fp+tn)
  return accuracy,prescion,f1,recall,tpr,fpr,tp,tn,fn,fp
print(knn_caller(data_trr,data_ttt,40,1))

In [ ]:
#GENERAL PARZEN WINDOW 
# checks if point2 lies in window around point
def valid(point,point2,parameter):
  n = len(parameter)
  for i in range(n):
    if abs(int(point2[i]) - int(point[i]))/parameter[i] > 0.5:
      return 0
  return 1

# Returns label for point given train data(Data) and window size (Parameter)
def parzen(data,point,parameter,th): 
  n = len(data)
  o = len(parameter)
  probab = [0,0]
  for i in range(n):
    probab[data[i][o]]+=valid(point,data[i],parameter)
  if probab[0]==0 or probab[1]/probab[0] > th:
    return 1
  else:
    return 0

#Evaluates over entire test set and returns the result parameters
def parzen_caller(data,test,parameter,th):
  n = len(test)
  o = len(data[0])
  tp = 0
  tn = 0
  fp = 0
  fn = 0
  right = 0
  for i in range(n):
    temp = parzen(data,test[i],parameter,th)
    if temp==test[i][o-1]:
      right+=1
    if temp==1 and test[i][o-1]==1:
      tp+=1
    elif temp==0 and test[i][o-1]==0:
      tn+=1
    elif temp!=0 and test[i][o-1]==0:
      fn+=1
    else:
      fp+=1
  prescion = tp/(tp+fp)
  recall = tp/(tp+fn)
  f1 = 2/(1/recall+1/prescion)
  accuracy = right/n
  tpr = tp/(tp+fn)
  fpr = fp/(fp+tn)
  return accuracy,prescion,f1,recall,tpr,fpr,tp,tn,fn,fp
parzen_caller(data_trr,data_ttt,[43,43,43],1)

In [ ]:
#---------------------------Linear kernel-----------------------------------------
# Genearates Nth order terms for a given order
lisss=['1','2','3']
def kernel(data,order):
  term_vec = kernel_gen(lisss,order)
  aa=np.array(data)
  kernel_data=[]
  dataT=aa.T
  for i in term_vec:
    c=dataT[int(i[0])-1]
    for j in range(1,len(i)):
      c=c*dataT[int(i[j])-1]
    kernel_data.append(c)
  return np.array(kernel_data).T

def kernel_gen(parameter,order):
  lis = ['']
  for iter in range(order):
    lis_2 = []
    for j in parameter:
      for k in lis:
        lis_2.append(k+j)
    for i in lis_2:
      ss=[]
      ss[:0]=i 
      ss.sort()
      ss="".join(ss)
      if ss not in lis:
        lis.append(ss)
  lis = lis[1:]
  return lis

In [ ]:
def linear_regr_reg_MAE_elasNet_vectorized(X,y,alpha,lambda1,lambda2,max_iter=10000):
  m,n=X.shape
  W=np.array( np.random.uniform(low=0.001,high=1, size=(X[0].shape[0],1)) )
  w_0=0
  y =  np.reshape(y,(y.shape[0],1))
  W=np.reshape(W,(n,1))
  for xxxxx in range(max_iter):
    Y_p=X.dot(W)+w_0
    # print(W.shape,Y_p.shape)
    Y_p=np.reshape(Y_p,(Y_p.shape[0],1))
    grad_W=np.zeros((n,1))
    grad_w_0=np.sum(np.where(Y_p-y>0,1,-1))/m
    ww=np.where(W>0,lambda1,-lambda1)
    # print(ww.shape)
    XX=np.where(Y_p-y>0,np.abs(X),-np.abs(X))
    grad_W=(np.reshape(np.sum(XX,axis=0),(n,1))+ww+2*lambda2*W)/m
    W=W-alpha*grad_W
    w_0=w_0-alpha*grad_w_0
    if abs(grad_w_0) < 0.001:
      break
  return W,w_0
models = np.zeros(1)
w0 = 0
def predict_linear_rgr(x,th):
  if x@model + w0 < th:
    return 0
  else:
    return 1
    
def eval12(data_tt,th,order):
  n = 0
  tp = 0
  tn = 0
  fp = 0
  fn = 0
  right = 0
  tot = 0
  for i in data_tt.items():
    ss = kernel(i[1],order)
    for j in ss:
      tot+=1
      temp = int(predict_linear_rgr(j,th))
      if temp==i[0]:
        right+=1
      if temp==1 and i[0]==1:
        tp+=1
      elif temp==0 and i[0]==0:
        tn+=1
      elif temp!=0 and i[0]==0:
        fn+=1
      else:
        fp+=1
  n = tot
  prescion = tp/(tp+fp)
  recall = tp/(tp+fn)
  f1 = 2/(1/recall+1/prescion)
  accuracy = right/n
  tpr = 1
  if tp + fn != 0:
    tpr = tp/(tp+fn)
  fpr = 1
  if fp + tn != 0:
    fpr = fp/(fp+tn)
  return accuracy,prescion,f1,recall,tpr,fpr,tp,tn,fn,fp

# caller
model,w0 =  linear_regr_reg_MAE_elasNet_vectorized(kernel(X,2),y,0.01,0,0)
print(eval12(data_tt,0.5,2))

In [ ]:
# Linear Regression with MSE elastic Net
# Trains and outputs models 
def linear_regr_MSE_elasnet_vec(X,y,alpha,lambda1,lambda2,max_iter=100000):
    m,n=X.shape
    W=np.array( np.random.uniform(low=0.001,high=1, size=(X[0].shape[0],1)) )
    w_0=0
    y = np.reshape(y,(y.shape[0],1))
    # print(y.shape)
    for i in range(max_iter):
        Y_pr=X.dot(W)+w_0
        WW=np.where(W>0,lambda1,-lambda1)
        grad_W=(-2*X.T@(y-Y_pr) +WW+2*lambda2*W)/m
        grad_w_0=-2*np.sum(y-Y_pr)/m
        W=W-alpha*grad_W
        w_0-=alpha*grad_w_0
        if(abs(grad_w_0)<1e-7):
          break
    return (W,w_0)

# predicts output for a particalar Sample witha a given THreshold
def predict_loge1t(x,th):
  if x@model + w_0 < th:
    return 0
  else:
    return 1
    
# Evaluvates all testset and returns relevent results
def eval2(data_tt,th,order):
  n = 0
  tp = 0
  tn = 0
  fp = 0
  fn = 0
  right = 0
  tot = 0
  for i in data_tt.items():
    ss = kernel(i[1],order)
    for j in ss:
      tot+=1
      temp = int(predict_loge1t(j,th))
      if temp==i[0]:
        right+=1
      if temp==1 and i[0]==1:
        tp+=1
      elif temp==0 and i[0]==0:
        tn+=1
      elif temp!=0 and i[0]==0:
        fn+=1
      else:
        fp+=1
  n = tot
  prescion = tp/(tp+fp)
  recall = tp/(tp+fn)
  f1 = 2/(1/recall+1/prescion)
  accuracy = right/n
  tpr = 1
  if tp + fn != 0:
    tpr = tp/(tp+fn)
  fpr = 1
  if fp + tn != 0:
    fpr = fp/(fp+tn)
  return accuracy,prescion,f1,recall,tpr,fpr,tp,tn,fn,fp
model,w_0 =  linear_regr_MSE_elasnet_vec(kernel(X,2),y,0.1,0,0)
print(eval2(data_tt,0.5,2))

In [ ]:
# logestic regression with all regularizer cross entropy

# some data preproceesing (adding 1 toa acount for w0)
def data_magic(X):
  nupea = []
  for i in range(len(X)):
    nupea.append( np.append(X[i],[1]) )
  return np.array(nupea)
  
# trains a logetistic regression for one label (one vs all used)
def magic(X,y,alpha,max_iter,lambda1,lambda2):
  X = data_magic(X)
  m=X.shape[0]
  n=X.shape[1]

  w=np.random.rand(X.shape[1])
  
  for i in range(max_iter):
    grad=X.T@(sigmoid(X@w)-y)/m
    for j in range(n-1):
      if w[j]>0:
        grad[j]+=(lambda1+2*lambda2*w[j])/m
      else:
        grad[j]+=(-lambda1+2*lambda2*w[j])/m
    w=w-alpha*grad 
  return w

# predicts label for a new input
def predict_loget(x,th):
  if sigmoid(np.append(x,1)@model) < th:
    return 0
  else:
    return 1

#evaluates entire train set and returns relevant results for logisitic regression
def eval1(data_tt,th,order):
  n = 0
  tp = 0
  tn = 0
  fp = 0
  fn = 0
  right = 0
  tot = 0
  for i in data_tt.items():
    ss = kernel(i[1],order)
    for j in ss:
      tot+=1
      temp = int(predict_loget(j,th))
      if temp==i[0]:
        right+=1
      if temp==1 and i[0]==1:
        tp+=1
      elif temp==0 and i[0]==0:
        tn+=1
      elif temp!=0 and i[0]==0:
        fn+=1
      else:
        fp+=1
  n = tot
  prescion = tp/(tp+fp)
  recall = tp/(tp+fn)
  f1 = 2/(1/recall+1/prescion)
  accuracy = right/n
  tpr = 1
  if tp + fn != 0:
    tpr = tp/(tp+fn)
  fpr = 1
  if fp + tn != 0:
    fpr = fp/(fp+tn)
  return accuracy,prescion,f1,recall,tpr,fpr,tp,tn,fn,fp
model =  magic(kernel(X,2),y,0.1,30000,0,0)
print(eval1(data_tt,0.5,2))

In [ ]:
# code for ROC and bais varaince and other things (Helper Functions)
results = []
roc = []
for i in range(1,40):
  results.append((knn_caller(data_trr,data_ttt,40,(2**i)/1e10)))
roc = []
test_loss = []
train_loss = []
for i in range(0,10):
    gmmtrain(1)
    oo = check_gmm(1,(10**i)/1e2)
    print(oo)
    roc.append([oo[4],oo[5]])
    test_loss.append(oo[0])
    train_loss.append(oo[6])
roc = []
for i in results:
    roc.append((i[4],i[5]))
roc = np.array(roc)
plt.plot(roc.T[1],roc.T[0])
plt.xlim(-0.01,1)
plt.ylim(0,1)